In [3]:
import os
import re
from PyPDF2 import PdfReader
from openai import OpenAI
from dotenv import load_dotenv
import json
from concurrent.futures import ThreadPoolExecutor

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def find_pdf_in_project():
    for root, dirs, files in os.walk(os.getcwd()):
        for file in files:
            if file.endswith('.pdf'):
                return os.path.join(root, file)
    return None

def extraire_sections(pdf_path, output_dir="data/sections"):
    """Extrait les sections d'un PDF et enregistre les métadonnées dans un fichier JSON.

    Args:
      pdf_path: Chemin d'accès au fichier PDF.
      output_dir: Répertoire de sortie pour les fichiers JSON.
    """
    os.makedirs(output_dir, exist_ok=True)
    reader = PdfReader(pdf_path)
    num_pages = len(reader.pages)

    section_texte = ""
    section_num = None
    section_metadata = []

    with ThreadPoolExecutor() as executor:
        for page_num in range(num_pages):
            try:
                page = reader.pages[page_num]
                texte = page.extract_text()

                if texte:
                    # Diviser le texte en lignes
                    lignes = texte.splitlines()
                    for ligne in lignes:
                        ligne = ligne.strip()
                        if ligne.isdigit():  # Vérifier si la ligne contient uniquement un chiffre
                            if section_num is not None:
                                # Ajouter les métadonnées de la section
                                section_metadata.append({
                                    'section_number': section_num,
                                    'page_number': page_num + 1
                                })
                            section_num = int(ligne)
                            section_texte = f"# Section {section_num}\n\n"
                        else:
                            # Ajouter la ligne à la section courante
                            if section_num is not None:
                                section_texte += ligne + "\n"

            except Exception as e:
                print(f"Erreur lors du traitement de la page {page_num + 1}: {e}")

        # Ajouter les métadonnées de la dernière section
        if section_num is not None:
            section_metadata.append({
                'section_number': section_num,
                'page_number': page_num + 1
            })

    # Sauvegarder les métadonnées dans un fichier JSON
    metadata_path = os.path.join(output_dir, 'section_metadata.json')
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(section_metadata, f, ensure_ascii=False, indent=4)
    print(f'Métadonnées sauvegardées dans: {metadata_path}')

# Exemple d'utilisation
pdf_path = find_pdf_in_project()
if pdf_path:
    print(f'PDF trouvé: {pdf_path}')
    extraire_sections(pdf_path)
else:
    print('Aucun PDF trouvé dans le projet')

PDF trouvé: c:\Users\erpes\Desktop\script extract chapters\Defis Fantastiques 18 - La Planete Rebelle.pdf
Métadonnées sauvegardées dans: data/sections\section_metadata.json
